In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/gender_submission.csv
/kaggle/input/titanic/test.csv


In [2]:
train_data = pd.read_csv("../input/titanic/train.csv")
test_data = pd.read_csv("../input/titanic/test.csv")
train_target = train_data['Survived']
train_data = train_data.drop(['Name','Survived','PassengerId','Ticket','Cabin','Embarked'], axis =1)
test_data_passengerId = test_data['PassengerId']
test_data = test_data.drop(['Name','Ticket','PassengerId','Cabin','Embarked'], axis =1)
train_data['Age']=train_data['Age'].fillna(train_data['Age'].mean())
test_data['Age']=test_data['Age'].fillna(test_data['Age'].mean())
test_data['Fare']=test_data['Fare'].fillna(test_data['Fare'].mean())
train_data = train_data.replace({'Sex':{'male':0 , 'female': 1}})
test_data = test_data.replace({'Sex':{'male':0 , 'female': 1}}) 

In [3]:
X_train,X_test,y_train,y_test = train_test_split(train_data, train_target, test_size = 0.2, random_state = 42)

# train_data , test_data , train_target , test_data_passengerId

In [4]:
train_data.head()

,Pclass,Sex,Age,SibSp,Parch,Fare
0,3,0,22.0,1,0,7.2500
1,1,1,38.0,1,0,71.2833
2,3,1,26.0,0,0,7.9250
3,1,1,35.0,1,0,53.1000
4,3,0,35.0,0,0,8.0500


In [5]:
test_data.head()

,Pclass,Sex,Age,SibSp,Parch,Fare
0,3,0,34.5,0,0,7.8292
1,3,1,47.0,1,0,7.0000
2,2,0,62.0,0,0,9.6875
3,3,0,27.0,0,0,8.6625
4,3,1,22.0,1,1,12.2875


In [6]:
train_target.head()

0    0
1    1
2    1
3    1
4    0
Name: Survived, dtype: int64

In [7]:
test_data.isnull().sum()

Pclass    0
Sex       0
Age       0
SibSp     0
Parch     0
Fare      0
dtype: int64

In [8]:
train_data.isnull().sum()

Pclass    0
Sex       0
Age       0
SibSp     0
Parch     0
Fare      0
dtype: int64

from sklearn.ensemble import RandomForestClassifier
rnd_clf = RandomForestClassifier(n_estimators=2000, max_leaf_nodes=200,max_features='auto', n_jobs=-1, min_samples_leaf=10,random_state =1, oob_score=True)
rnd_clf.fit(train_data,train_target) 
print(rnd_clf.oob_score_)
y_pred_rf = rnd_clf.predict(test_data)

from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingClassifier
num_estimators = [500,1000]
learn_rates = [0.02, 0.05]
max_depths = [1, 2]
min_samples_leaf = [5,10]
min_samples_split = [5,10]

param_grid = {'n_estimators': num_estimators,
              'learning_rate': learn_rates,
              'max_depth': max_depths,
              'min_samples_leaf': min_samples_leaf,
              'min_samples_split': min_samples_split}

random_search = RandomizedSearchCV(GradientBoostingClassifier(loss="deviance"), param_grid, random_state=1, n_iter=100, cv=5, verbose=0, n_jobs=-1)

random_search.fit(train_data,train_target)

random_search.best_params_

gboost_score=random_search.score(train_data,train_target)
print(gboost_score)

In [9]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier

num_estimators = [500,1000,1500]
learn_rates = [0.02, 0.05,0.08,0.1]
max_depths = [1, 2, 3]
min_samples_leaf = [5,10,15]
min_samples_split = [5,10,15]

param_grid = {'n_estimators': num_estimators,
              'learning_rate': learn_rates,
              'max_depth': max_depths,
              'min_samples_leaf': min_samples_leaf,
              'min_samples_split': min_samples_split}

grid_search = GridSearchCV(GradientBoostingClassifier(loss="exponential"),
                           param_grid, cv=3, return_train_score=True)
grid_search = grid_search.fit(X_train,y_train)
predictions = grid_search.predict(X_test)
accuracy_score(y_true = y_test, y_pred = predictions)

0.8435754189944135

grid_search.best_params_

grid_gboost_result = grid_search.predict(test_data)

random_gboost_result = random_search.predict(test_data)

grid_score = grid_search.score(train_data,train_target)
print(grid_score)

output = pd.DataFrame({'PassengerId': test_data_passengerId, 'Survived': grid_gboost_resul})
output.to_csv('my_submission_GGB.csv',index=False)

output2 = pd.DataFrame({'PassengerId': test_data_passengerId, 'Survived': random_gboost_result})
output2.to_csv('my_submission_RGB.csv',index=False)

output3 = pd.DataFrame({'PassengerId': test_data_passengerId, 'Survived': y_pred_rf})
output3.to_csv('my_submission_RF.csv',index=False)

output.head(10)

output3.head(10)